This notebook showcases an agent designed to interact with a sql databases. The agent builds off of SQLDatabaseChain and is designed to answer more general questions about a database, as well as recover from errors.

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

In [ ]:
import sqlite3

# Connect to a new database file called "mydatabase.db"
conn = sqlite3.connect('Chinook.db')

# Close the database connection
conn.close()


In [ ]:
import sqlite3

# Connect to a new database file
conn = sqlite3.connect('Chinook.db')

# Open the SQL script file and read its contents
with open('./Chinook_Sqlite.sql', 'r') as f:
    sql_script = f.read()

# Execute the SQL script in the new database
conn.executescript(sql_script)

# Close the database connection
conn.close()

In [ ]:
import sqlite3

# Connect to an existing database file
conn = sqlite3.connect('Chinook.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query
cursor.execute('SELECT * FROM Artist LIMIT 10')

# Fetch the query results
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and database connection
cursor.close()
conn.close()


(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ""


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

In [ ]:
# db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
# toolkit = SQLDatabaseToolkit(db=db)

# agent_executor = create_sql_agent(
#     llm=OpenAI(temperature=0),
#     toolkit=toolkit,
#     verbose=True
# )


from langchain_community.llms import OpenAI  # Assuming OpenAI is imported from langchain_community

# Assuming db and toolkit initialization remains the same
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)


# Example: running queries

In [ ]:
agent_executor.run("List the total sales per country.")



> Entering new SQL Agent Executor chain...
 We need to query the database for the total sales and group them by country.
Action: sql_db_query
Action Input: SELECT country, SUM(sales) FROM sales_table GROUP BY countryError: (sqlite3.OperationalError) no such table: sales_table
[SQL: SELECT country, SUM(sales) FROM sales_table GROUP BY country]
(Background on this error at: https://sqlalche.me/e/20/e3q8)We need to check the table name and make sure it exists in the database.
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track The sales table is not listed, we need to check the schema to find the correct table name.
Action: sql_db_schema
Action Input: sales_tableError: table_names {'sales_table'} not found in database We need to check the schema for all tables and find the correct table name for sales.
Action: sql_db_schema
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, Inv

'The total sales per country are: Argentina - $37.62, Australia - $37.62, Austria - $42.62, Belgium - $37.62, Brazil - $190.10, Canada - $303.96, Chile - $46.62, Czech Republic - $90.24, Denmark - $37.62, Finland - $41.62, France - $195.10, Germany - $156.48, Hungary - $45.62, India - $75.26, Ireland - $45.62, Italy - $37.62, Netherlands - $40.62, Norway - $39.62, Poland - $37.62, Portugal - $77.24, Spain - $37.62, Sweden - $38.62, USA - $523.06, United Kingdom - $112.86.'

In [ ]:
agent_executor.run("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")



> Entering new SQL Agent Executor chain...
 I need to query the database to get the total number of tracks in each playlist. I also need to include the playlist name in the result.
Action: sql_db_query
Action Input: SELECT Playlist.Name, COUNT(Track.TrackId) FROM Playlist INNER JOIN PlaylistTrack ON Playlist.PlaylistId = PlaylistTrack.PlaylistId INNER JOIN Track ON PlaylistTrack.TrackId = Track.TrackId GROUP BY Playlist.Name[('90’s Music', 1477), ('Brazilian Music', 39), ('Classical', 75), ('Classical 101 - Deep Cuts', 25), ('Classical 101 - Next Steps', 25), ('Classical 101 - The Basics', 25), ('Grunge', 15), ('Heavy Metal Classic', 26), ('Music', 6580), ('Music Videos', 1), ('On-The-Go 1', 1), ('TV Shows', 426)]I need to make sure the query is correct before executing it.
Action: sql_db_query_checker
Action Input: SELECT Playlist.Name, COUNT(Track.TrackId) FROM Playlist INNER JOIN PlaylistTrack ON Playlist.PlaylistId = PlaylistTrack.PlaylistId INNER JOIN Track ON PlaylistTrack.Trac

'The total number of tracks in each playlist are:\n90’s Music - 1477\nBrazilian Music - 39\nClassical - 75\nClassical 101 - Deep Cuts - 25\nClassical 101 - Next Steps - 25\nClassical 101 - The Basics - 25\nGrunge - 15\nHeavy Metal Classic - 26\nMusic - 6580\nMusic Videos - 1\nOn-The-Go 1 - 1\nTV Shows - 426'

# Recovering from an error
In this example, the agent is able to recover from an error after initially trying to access an attribute (Track.ArtistId) which doesn’t exist.

In [ ]:
agent_executor.run("Who are the top 3 best selling artists?")



> Entering new SQL Agent Executor chain...
 We need to query the database to get the top 3 best selling artists.
Action: sql_db_query
Action Input: SELECT artist_name, SUM(sales) AS total_sales FROM artists GROUP BY artist_name ORDER BY total_sales DESC LIMIT 3Error: (sqlite3.OperationalError) no such table: artists
[SQL: SELECT artist_name, SUM(sales) AS total_sales FROM artists GROUP BY artist_name ORDER BY total_sales DESC LIMIT 3]
(Background on this error at: https://sqlalche.me/e/20/e3q8)We need to check the schema of the database to see if there is a table called "artists".
Action: sql_db_schema
Action Input: artistsError: table_names {'artists'} not found in database We need to list all the tables in the database to see if there is a table called "artists".
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track We now know that there is a table called "artists" in the database.
Action:

'The top 3 best selling artists are Wilhelm Kempff, Santana Feat. Eagle-Eye Cherry, and Royal Philharmonic Orchestra & Sir Thomas Beecham.'

In [ ]:
agent_executor("Which Artist recorded the longest track?")



> Entering new SQL Agent Executor chain...
 I need to find the track length and artist name from the database.
Action: sql_db_query
Action Input: SELECT artist_name, track_length FROM tracks ORDER BY track_length DESC LIMIT 1Error: (sqlite3.OperationalError) no such table: tracks
[SQL: SELECT artist_name, track_length FROM tracks ORDER BY track_length DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)I need to use sql_db_list_tables to see which tables are in the database.
Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI need to use sql_db_schema to see the schema for the Track table.
Action: sql_db_schema
Action Input: Track
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEG

{'input': 'Which Artist recorded the longest track?',
 'output': 'The artist with the longest track is Angus Young, Malcolm Young, and Brian Johnson with their track "For Those About To Rock (We Salute You)" at 343719 milliseconds.'}